In [1]:
#Web scraping imports 
from bs4 import BeautifulSoup as bs 
from selenium import webdriver
import time 

#Random import 
import numpy as np

#DataFrame import
import pandas as pd

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
#Dictionary for storage
listings_dict = {'Price (usd)' : [], "Address" : [], 'Rooms' : [], 'Bathrooms' : [], 'Area (sqrt)' : [], 'Link' : [], "typeProperty" : [], "yearBuilt" : [], "parking" : [], "lot" : [], "saleHistory" : [], "taxHistory" : [] }

#Path to drivers - need 2 browsers 1 for the main page listing and another for the individual links of each linsting.
# If this is not done and we do everything in the same browser Zillow detects the crawler. 
# Remember to unzip the file of the drivers and get the absolute path to the driver not the relative path.
#In the folder, copy the chrome driver and rename the copy as chromedriver2. Make sure browser is linked to chromedriver AND browser2 is linked to chromedriver2
browser = webdriver.Chrome("/Users/Sebastian/Documents/GitHub/Data Visualization Bootcamp/Sebastian Homework/TFH-Project2/Zillow Scrapper/chromedriver")
browser2 = webdriver.Chrome("/Users/Sebastian/Documents/GitHub/Data Visualization Bootcamp/Sebastian Homework/TFH-Project2/Zillow Scrapper/chromedriver2")

#starting page
page = 1 

#url to scrape starting at page 1
url = "https://www.zillow.com/homes/Austin,-TX_rb/"

#Open url -- We only want to open the browser once, so this is run outside the loop.
browser.get(url)

#while loop to iterate until condition is meet
while True:

    #Bot handler. -- Gives us 5 minutes to solve the human test and continues the code after it sees class = list-card-info.
    try:
        WebDriverWait(browser, 60*5).until(EC.presence_of_element_located((By.CLASS_NAME, "list-card-info")))
    except:
        print("Wait Timed out")

    

    #Prints current page to scrape
    print(f"Current page: {page}")

    #Random lag generator - try to keep the code to be detected as a bot 
    lag = np.random.uniform(0, 2*60)
    time.sleep(lag)

    #save the html source of the main page.
    html = browser.page_source

    #Use bs4 to parse the html response.
    soup = bs(html, "html.parser")

    #Results for all the listings in the page.
    listings = soup.find_all('div', class_="list-card-info")

    #For loop to evaluate each listing per page
    for listing in listings:

        #page index counter
        count = 0

        #Defines each variable for each iteration
        price = None 
        address = None 
        rooms = None 
        bathrooms = None 
        area = None 
        link = None 
        typeProperty = None 
        yearBuilt = None 
        parking = None 
        lot = None 
        saleHistory = None 
        taxHistory = None

        #Try the main page.
        try:
            #Price of the property
            try:
                price = int(listing.find('div', class_="list-card-price").text.replace("$","").replace(",",""))
            except ValueError as e:
                print(f"{e} error at price at {count}")

            #Address
            try:
                address = listing.find('address', class_="list-card-addr").text
            except ValueError as e:
                print(f"{e} error at address at {count}")

            #Rooms  - some are listed as Studio, bds and bd
            try:
                try:
                    rooms = int(listing.find_all('li')[0].text.replace(" bds",""))
                except ValueError as e:
                    pass
                try:
                    rooms = int(listing.find_all('li')[0].text.replace("Studio","1"))
                except ValueError as e:
                    pass
                try:
                    rooms = int(listing.find_all('li')[0].text.replace(" bd",""))
                except ValueError as e:
                    pass
            except ValueError as e:
                print(f"{e} error at rooms at {count}")

            #Bathrooms
            try:
                bathrooms = int(listing.find_all('li')[1].text.replace(" ba",""))
            except ValueError:
                if (listing.find_all('li')[1].text == '-- ba'):
                    continue

            #Area of the house.
            try:
                area = int(listing.find_all('li')[2].text.replace(" sqft","").replace(",",""))
            except ValueError as e:
                print(f"{e} error at area at {count}")

            #link of the individual listing.
            try:
                link = listing.a['href']
            except ValueError as e:
                print(f"{e} error at rooms at {count}")

            #Opens the listing link using the second browser
            browser2.get(link)

            #Bot handler. -- Gives us 5 minutes to solve the human test and continues the code after it sees class = ds-body.
            try:
                WebDriverWait(browser2, 60*5).until(EC.presence_of_element_located((By.CLASS_NAME, "ds-body")))
            except:
                print("Wait Timed out")

            #Random lag generator - try to keep the code to be detected as a bot 
            lag = np.random.uniform(0, 3*60)
            time.sleep(lag)

            #Save the html of the listing link. 
            htmlink = browser2.page_source

            #Use bs4 to parse the html response of the listing link
            souplink = bs(htmlink, "html.parser")

            #Type of property
            try: 
                typeProperty = souplink.find_all('span', class_="ds-body ds-home-fact-value")[0].text
            except ValueError as e:
                print(f"{e}Error at type property at {link}")
            
            #Year
            try:
                yearBuilt = int(souplink.find_all('span', class_="ds-body ds-home-fact-value")[1].text)
            except ValueError as e:
                if (souplink.find_all('span', class_="ds-body ds-home-fact-value")[1].text == 'No Data'):
                    continue
                print(f"{e}Error at yearbuilt at {link}")

            #Parking - listed as spaces, space, detached garage, off street, carport
            try:
                try:
                    parking = int(souplink.find_all('span', class_="ds-body ds-home-fact-value")[4].text.replace(" spaces",""))
                except ValueError:
                    pass
                try:
                    parking = int(souplink.find_all('span', class_="ds-body ds-home-fact-value")[4].text.replace(" space",""))
                except ValueError:
                    pass
                try:
                    parking = int(souplink.find_all('span', class_="ds-body ds-home-fact-value")[4].text.replace("Detached Garage","1"))
                except ValueError:
                    pass
                try:
                    parking = int(souplink.find_all('span', class_="ds-body ds-home-fact-value")[4].text.replace("Off street","1"))
                except ValueError:
                    pass
                try:
                    parking = int(souplink.find_all('span', class_="ds-body ds-home-fact-value")[4].text.replace("Carport","1"))
                except ValueError:
                    pass
            except ValueError as e:
                print(f"{e} garage error not determined at index {count}")
    
            #Lot - if the house has HOW the listing will move. listed as sqft and acres.
            try:
                try: 
                    lot = int(souplink.find_all('span', class_="ds-body ds-home-fact-value")[5].text.replace(" sqft","").replace(",",""))
                except ValueError:
                    pass 
                try:
                    lot = int(souplink.find_all('span', class_="ds-body ds-home-fact-value")[6].text.replace(" sqft","").replace(",",""))
                except ValueError:
                    pass
                try:
                    lot = int(souplink.find_all('span', class_="ds-body ds-home-fact-value")[5].text.replace(" acres","").replace(",","")) * 43560
                except ValueError:
                    pass
                try:
                    lot = int(souplink.find_all('span', class_="ds-body ds-home-fact-value")[6].text.replace(" acres","").replace(",","")) * 43560
                except ValueError:
                    pass

            except (ValueError, IndexError) as e:
                print(f"{e} error at lot type at {link}")

            #Sale history
            saleHistory = []
            for i in souplink.find_all('td', class_="kaq65t-1 ijBExC"):
                try:
                    saleHistory.append(i.find('span').text)
                except AttributeError:
                    continue 

            #tax history
            taxHistory = []
            for i in souplink.find_all('td', class_="kaq65t-1 eegcAA"):
                try:
                    taxHistory.append(i.find('span').text)
                except AttributeError:
                    continue

            #Only appends listings with complete information
            if (price and address and rooms and bathrooms and area and link and typeProperty and yearBuilt and parking and lot and saleHistory and taxHistory):

                listingInformation = [price, address, rooms, bathrooms, area, link, typeProperty, yearBuilt, parking, lot, saleHistory, taxHistory]

                #For loop to append the scraped information.
                for key,info in zip(listings_dict.keys(), listingInformation):
                    listings_dict[key].append(info)

            #Adds 1 to the count at the end of the listing scrape
            count += 1
            
        except ValueError as e:
            print(f"{e} error at index {count} page {page}")
            count += 1

    #Adds 1 to the page counter
    page += 1

    #Random lag generator - try to keep the code to be detected as a bot 
    lag = np.random.uniform(0, 1.5*60)
    time.sleep(lag)

    #Finds elements to go next page
    nextpage = browser.find_element_by_css_selector("a.Button-wpcbcc-0.lcWnHB.PaginationButton-si2hz6-0.cUjspl")

    #Zillow only loads 20 page (500 listings) at  time.
    if page <= 20:
        #clicks next page.
        nextpage.click()

    else:
        #When the code is done, this code kills the driver.
        browser.quit()
        browser2.quit()
        print("Scape Done!!!!")
        break

Current page: 1


KeyboardInterrupt: 

In [ ]:
#Dictionary for storage
listings_dict = {'Price (usd)' : [], "Address" : [], 'Rooms' : [], 'Bathrooms' : [], 'Area (sqrt)' : [], 'Link' : []}

#Path to drivers - need 2 browsers 1 for the main page listing and another for the individual links of each linsting.
# If this is not done and we do everything in the same browser Zillow detects the crawler. 
# Remember to unzip the file of the drivers and get the absolute path to the driver not the relative path.
#In the folder, copy the chrome driver and rename the copy as chromedriver2. Make sure browser is linked to chromedriver AND browser2 is linked to chromedriver2
browser = webdriver.Chrome("/Users/Sebastian/Documents/GitHub/Data Visualization Bootcamp/Sebastian Homework/TFH-Project2/Zillow Scrapper/chromedriver")
#browser2 = webdriver.Chrome("/Users/Sebastian/Documents/GitHub/Data Visualization Bootcamp/Sebastian Homework/TFH-Project2/Zillow Scrapper/chromedriver2")

#starting page
page = 1 

#url to scrape starting at page 1
url = "https://www.zillow.com/homes/Austin,-TX_rb/"

#Open url -- We only want to open the browser once, so this is run outside the loop.
browser.get(url)

#while loop to iterate until condition is meet
while True:

    #Bot handler. -- Gives us 5 minutes to solve the human test and continues the code after it sees class = list-card-info.
    try:
        WebDriverWait(browser, 60*5).until(EC.presence_of_element_located((By.CLASS_NAME, "list-card-info")))
    except:
        print("Wait Timed out")

    

    #Prints current page to scrape
    print(f"Current page: {page}")

    #Random lag generator - try to keep the code to be detected as a bot 
    lag = np.random.uniform(0, 1.5*60)
    time.sleep(lag)

    #save the html source of the main page.
    html = browser.page_source

    #Use bs4 to parse the html response.
    soup = bs(html, "html.parser")

    #Results for all the listings in the page.
    listings = soup.find_all('div', class_="list-card-info")

    #For loop to evaluate each listing per page
    for listing in listings:

        #page index counter
        count = 0

        #Defines each variable for each iteration
        price = None 
        address = None 
        rooms = None 
        bathrooms = None 
        area = None 
        link = None 
        typeProperty = None 
        yearBuilt = None 
        parking = None 
        lot = None 
        saleHistory = None 
        taxHistory = None

        #Try the main page.
        try:
            #Price of the property
            try:
                price = int(listing.find('div', class_="list-card-price").text.replace("$","").replace(",",""))
            except ValueError as e:
                print(f"{e} error at price at {count}")

            #Address
            try:
                address = listing.find('address', class_="list-card-addr").text
            except ValueError as e:
                print(f"{e} error at address at {count}")

            #Rooms  - some are listed as Studio, bds and bd
            try:
                try:
                    rooms = int(listing.find_all('li')[0].text.replace(" bds",""))
                except ValueError as e:
                    pass
                try:
                    rooms = int(listing.find_all('li')[0].text.replace("Studio","1"))
                except ValueError as e:
                    pass
                try:
                    rooms = int(listing.find_all('li')[0].text.replace(" bd",""))
                except ValueError as e:
                    pass
            except ValueError as e:
                print(f"{e} error at rooms at {count}")

            #Bathrooms
            try:
                bathrooms = int(listing.find_all('li')[1].text.replace(" ba",""))
            except ValueError:
                if (listing.find_all('li')[1].text == '-- ba'):
                    continue

            #Area of the house.
            try:
                area = int(listing.find_all('li')[2].text.replace(" sqft","").replace(",",""))
            except ValueError as e:
                print(f"{e} error at area at {count}")

            #link of the individual listing.
            try:
                link = listing.a['href']
            except ValueError as e:
                print(f"{e} error at rooms at {count}")

            # #Opens the listing link using the second browser
            # browser2.get(link)

            # #Bot handler. -- Gives us 5 minutes to solve the human test and continues the code after it sees class = ds-body.
            # try:
            #     WebDriverWait(browser2, 60*5).until(EC.presence_of_element_located((By.CLASS_NAME, "ds-body")))
            # except:
            #     print("Wait Timed out")

            # #Random lag generator - try to keep the code to be detected as a bot 
            # lag = np.random.uniform(0, 3*60)
            # time.sleep(lag)

            # #Save the html of the listing link. 
            # htmlink = browser2.page_source

            # #Use bs4 to parse the html response of the listing link
            # souplink = bs(htmlink, "html.parser")

            # #Type of property
            # try: 
            #     typeProperty = souplink.find_all('span', class_="ds-body ds-home-fact-value")[0].text
            # except ValueError as e:
            #     print(f"{e}Error at type property at {link}")
            
            # #Year
            # try:
            #     yearBuilt = int(souplink.find_all('span', class_="ds-body ds-home-fact-value")[1].text)
            # except ValueError as e:
            #     if (souplink.find_all('span', class_="ds-body ds-home-fact-value")[1].text == 'No Data'):
            #         continue
            #     print(f"{e}Error at yearbuilt at {link}")

            # #Parking - listed as spaces, space, detached garage, off street, carport
            # try:
            #     try:
            #         parking = int(souplink.find_all('span', class_="ds-body ds-home-fact-value")[4].text.replace(" spaces",""))
            #     except ValueError:
            #         pass
            #     try:
            #         parking = int(souplink.find_all('span', class_="ds-body ds-home-fact-value")[4].text.replace(" space",""))
            #     except ValueError:
            #         pass
            #     try:
            #         parking = int(souplink.find_all('span', class_="ds-body ds-home-fact-value")[4].text.replace("Detached Garage","1"))
            #     except ValueError:
            #         pass
            #     try:
            #         parking = int(souplink.find_all('span', class_="ds-body ds-home-fact-value")[4].text.replace("Off street","1"))
            #     except ValueError:
            #         pass
            #     try:
            #         parking = int(souplink.find_all('span', class_="ds-body ds-home-fact-value")[4].text.replace("Carport","1"))
            #     except ValueError:
            #         pass
            # except ValueError as e:
            #     print(f"{e} garage error not determined at index {count}")
    
            # #Lot - if the house has HOW the listing will move. listed as sqft and acres.
            # try:
            #     try: 
            #         lot = int(souplink.find_all('span', class_="ds-body ds-home-fact-value")[5].text.replace(" sqft","").replace(",",""))
            #     except ValueError:
            #         pass 
            #     try:
            #         lot = int(souplink.find_all('span', class_="ds-body ds-home-fact-value")[6].text.replace(" sqft","").replace(",",""))
            #     except ValueError:
            #         pass
            #     try:
            #         lot = int(souplink.find_all('span', class_="ds-body ds-home-fact-value")[5].text.replace(" acres","").replace(",","")) * 43560
            #     except ValueError:
            #         pass
            #     try:
            #         lot = int(souplink.find_all('span', class_="ds-body ds-home-fact-value")[6].text.replace(" acres","").replace(",","")) * 43560
            #     except ValueError:
            #         pass

            # except (ValueError, IndexError) as e:
            #     print(f"{e} error at lot type at {link}")

            # #Sale history
            # saleHistory = []
            # for i in souplink.find_all('td', class_="kaq65t-1 ijBExC"):
            #     try:
            #         saleHistory.append(i.find('span').text)
            #     except AttributeError:
            #         continue 

            # #tax history
            # taxHistory = []
            # for i in souplink.find_all('td', class_="kaq65t-1 eegcAA"):
            #     try:
            #         taxHistory.append(i.find('span').text)
            #     except AttributeError:
            #         continue

            #Only appends listings with complete information
            if (price and address and rooms and bathrooms and area and link):

                listingInformation = [price, address, rooms, bathrooms, area, link]

                #For loop to append the scraped information.
                for key,info in zip(listings_dict.keys(), listingInformation):
                    listings_dict[key].append(info)

            #Adds 1 to the count at the end of the listing scrape
            count += 1
            
        except ValueError as e:
            print(f"{e} error at index {count} page {page}")
            count += 1

    #Adds 1 to the page counter
    page += 1

    

    #Finds elements to go next page
    nextpage = browser.find_element_by_css_selector("a.Button-wpcbcc-0.lcWnHB.PaginationButton-si2hz6-0.cUjspl")

    #Zillow only loads 20 page (500 listings) at  time.
    if page <= 20:
        #clicks next page.
        nextpage.click()
        #Random lag generator - try to keep the code to be detected as a bot 
        lag = np.random.uniform(0, 1.5*60)
        time.sleep(lag)

    else:
        #When the code is done, this code kills the driver.
        browser.quit()
        #browser2.quit()
        print("Scape Done!!!!")
        break

In [5]:
browser.quit()
browser2.quit()


In [4]:
listings_df = pd.DataFrame(listings_dict)

In [5]:
listings_df

,Price (usd),Address,Rooms,Bathrooms,Area (sqrt),Link,typeProperty,yearBuilt,heating,cooling,parking,lot,saleHistory,taxHistory
0,425000,"2804 Tinmouth St, Austin, TX 78748",4,3,2251,https://www.zillow.com/homedetails/2804-Tinmou...,Single Family,2006,Forced air,Central,2,7636,"[7/30/2020, Listed for sale, $425,000 ]","[2019, $6,377 , $336,252 , 2018, $6,055 , $313..."
1,345000,"5908 Jacob Gln, Austin, TX 78727",3,2,1089,https://www.zillow.com/homedetails/5908-Jacob-...,Single Family,1985,Forced air,Central,2,3963,"[7/30/2020, Listed for sale, $345,000 ]","[2019, $1,896 , $242,424 , 2018, $1,605 , $211..."
2,330000,"3225 Jewelfish Cv, Austin, TX 78728",3,2,1811,https://www.zillow.com/homedetails/3225-Jewelf...,Single Family,1999,Forced air,Central,2,6324,"[7/30/2020, Listed for sale, $330,000 , 2/27/2...","[2019, $5,926 , $309,510 , 2018, $5,477 , $298..."
3,419900,"2453 Rick Whinery Dr, Austin, TX 78728",3,3,2540,https://www.zillow.com/homedetails/2453-Rick-W...,Single Family,1995,Forced air,Central,2,6621,"[7/30/2020, Listed for sale, $419,900 ]","[2019, $6,277 , $359,783 , 2018, $6,155 , $321..."
4,1595000,"1605 Westover Rd, Austin, TX 78703",4,4,2832,https://www.zillow.com/homedetails/1605-Westov...,Single Family,1984,Forced air,Central,1,8276,"[7/30/2020, Listed for sale, $1,595,000 , 6/15...","[2019, $16,787 , $976,471 , 2018, $15,717 , $9..."
5,489900,"11610 Shoshone Dr, Austin, TX 78759",3,2,1651,https://www.zillow.com/homedetails/11610-Shosh...,Single Family,1973,Forced air,Central,2,7187,"[7/30/2020, Listed for sale, $489,900 ]","[2019, $7,312 , $378,820 , 2018, $7,279 , $369..."
6,450000,"3308 Oxsheer Dr, Austin, TX 78732",4,3,2752,https://www.zillow.com/homedetails/3308-Oxshee...,Single Family,1999,Forced air,Central,2,8755,"[7/30/2020, Listed for sale, $450,000 ]","[2019, $8,800 , $387,500 , 2018, $9,350 , $393..."
7,415000,"3514 Lafayette Ave, Austin, TX 78722",2,1,776,https://www.zillow.com/homedetails/3514-Lafaye...,Single Family,1946,Forced air,Central,1,5009,"[7/30/2020, Listed for sale, $415,000 ]","[2019, $7,384 , $344,268 , 2018, $8,180 , $361..."
8,339900,"2214 Marcus Abrams Blvd, Austin, TX 78748",3,3,1860,https://www.zillow.com/homedetails/2214-Marcus...,Single Family,2003,Forced air,Central,2,4965,"[7/30/2020, Listed for sale, $339,900 , 9/29/2...","[2019, $6,018 , $280,571 , 2018, $5,813 , $270..."
9,2250000,"1403 W 9th St, Austin, TX 78703",3,3,2666,https://www.zillow.com/homedetails/1403-W-9th-...,Single Family,1878,Forced air,Central,2,8712,"[7/30/2020, Listed for sale, $2,250,000 ]","[2019, $26,356 , $1,449,527 , 2018, -- , $1,09..."


In [6]:
listings_df.to_csv('zillowOuput.csv')